In [1]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from upsetplot import UpSet, from_contents, plot, from_indicators
import numpy as np
import seaborn as sns 
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings("ignore")

In [2]:
def read_csv_data(mfile):
    adata = pd.read_csv(mfile,sep=",",header=0)
    adata.columns = ['gene','C','Z','Pval','FDR','cluster']
    adata = adata[adata['cluster'] > 0].copy()
    adata['cluster'] = adata['cluster'].astype(int)  # 转换为整数
    return adata

#筛选出指定的cluster（3.17修改）
def find_specfic_cluster(data,target_cluster):
    filterd_df = data[data['cluster'].isin(target_cluster)]#选择target_cluster的行
    return filterd_df

#生成每个细胞类型的字典
def module_gene_dic(data,name):
    indicators_df= from_contents({key:df['gene'].tolist() for key,df in data.items()})
    upset = UpSet(indicators_df,sort_by='degree',subset_size='count',show_counts=True,min_degree=(len(data) - 4))
    upset.plot()

    test = indicators_df.reset_index().set_index('id')#将索引重置，即将原来的索引列（默认是index）转换为普通列，并生成一个新的默认索引。
    sss = test.sum(axis =1).reset_index()#按行求和
    sss.columns = ['gene','count']
    result = sss.sort_values('count',ascending=False)
    result['type'] = name
    return result

def ancestral_input_data(data,name,save = None):
    counts = len(data)/4
    indicators_df= from_contents({key:df['gene'].tolist() for key,df in data.items()})
    df= indicators_df.reset_index()
    # df.columns = ['gene','type']
    df =df.replace({True: 1, False: 0})
    df['counts'] = df.drop('id', axis=1).sum(axis=1)
    # 按counts降序排列
    df = df.sort_values(by='counts', ascending=False).reset_index(drop=True)
    df = df[df['counts'] >= counts]
    df['counts'] = df['counts'].astype(str) + ' (' + name + ')'
    if save:
        df.to_csv(save, index=False)
    df
    return df


In [3]:
csv_data_dic = {}
csv_data_dic['Ome.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/01.Oryzias_melastigma_intestine60.modules4.0.csv') 
csv_data_dic['Mat.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/02.Megalops_atlanticus_intestine60.modules4.0.csv') 
csv_data_dic['Psa.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/03.Polyodon_spathula_intestine70.modules4.0.csv' ) 
csv_data_dic['Pan.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/04.Protopterus_annectens_intestine60.modules4.0.csv')   
csv_data_dic['Pse.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/05.Polypterus_senegalus_intestine60.modules4.0.csv')
csv_data_dic['Cpl.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/06.Chiloscyllium_plagiosum_intestine60.modules4.0.csv') 
csv_data_dic['Lja.G'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/07.Lampetra_japonica_intestine40.modules4.0.csv')
csv_data_dic['Pbu.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/08.Pantodon_buchholzi_intestine80.modules4.0.csv')
csv_data_dic['Asp.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/09.Atractosteus_spatula_intestine60.modules4.0.csv')
csv_data_dic['Hsp.I'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/00.Homo_sapiens_intestine60.2.modules4.0.csv')

csv_data_dic['Ome.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/01.Oryzias_melastigma_stomach40.modules4.0.csv') 
csv_data_dic['Mat.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/02.Megalops_atlanticus_stomach60.2.modules4.0.csv') 
csv_data_dic['Psa.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/03.Polyodon_spathula_stomach60.modules4.0.csv') 
csv_data_dic['Pan.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/04.Protopterus_annectens_stomach60.modules4.0.csv')   
csv_data_dic['Pse.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/05.Polypterus_senegalus_stomach70.modules4.0.csv')
csv_data_dic['Cpl.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/06.Chiloscyllium_plagiosum_stomach60.modules4.0.csv') 
csv_data_dic['Pbu.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/08.Pantodon_buchholzi_stomach60.modules4.0.csv')
csv_data_dic['Asp.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/09.Atractosteus_spatula_stomach80.modules4.0.csv')
csv_data_dic['Hsp.S'] = read_csv_data('D:/project/digestion_SI/03.hotspot/modules2/00.Homo_sapiens_stomach60.2.modules4.0.csv') 

In [4]:
#Endothelial
Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[5])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[3])
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[4])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[4])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[14,3])
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[3])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[4])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[4])
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[3])
Hsp_I_L = find_specfic_cluster(csv_data_dic['Hsp.I'],[32,3])
Hsp_I_V = find_specfic_cluster(csv_data_dic['Hsp.I'],[3,25,33,24,22,28])

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[6])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[8])
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[5])
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[5])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[2])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[4])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[6,14])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[5])
Hsp_S_L = find_specfic_cluster(csv_data_dic['Hsp.S'],[2,28,15,16,25,9,18,22,11,13,19])
Hsp_S_V = find_specfic_cluster(csv_data_dic['Hsp.S'],[28,2,13])

Endothelial = {
    'Ome_I.Endothelial':Ome_I,
    'Mat_I.Endothelial':Mat_I,
    'Psp_I.Endothelial':Psa_I,
    'Pan_I.Endothelial':Pan_I,
    'Pse_I.Endothelial':Pse_I,
    'Cpl_I.Endothelial':Cpl_I,
    'Lre_G.Endothelial':Lja_G,
    'Pbu_I.Endothelial':Pbu_I,
    'Asp_I.Endothelial':Asp_I,
    'Hsa_I.Lymphatic_endothelia':Hsp_I_L,
    'Hsa_I.Vascular_endothelia':Hsp_I_V,
    
    'Ome_D.Endothelial':Ome_S,
    'Mat_S.Endothelial':Mat_S,
    'Psp_S.Endothelial':Psa_S,
    'Pan_D.Endothelial':Pan_S,
    'Pse_S.Endothelial':Pse_S,
    'Cpl_S.Endothelial':Cpl_S,
    'Pbu_S.Endothelial':Pbu_S,
    'Asp_S.Endothelial':Asp_S, 
    'Hsa_S.Lymphatic_endothelia':Hsp_S_L,
    'Hsa_S.Vascular_endothelia':Hsp_S_V,
}

In [5]:
Endothelial_ai = ancestral_input_data(Endothelial,'Endothelial')
Endothelial_ai

,Ome_I.Endothelial,Mat_I.Endothelial,Psp_I.Endothelial,Pan_I.Endothelial,Pse_I.Endothelial,Cpl_I.Endothelial,Lre_G.Endothelial,Pbu_I.Endothelial,Asp_I.Endothelial,Hsa_I.Lymphatic_endothelia,...,Psp_S.Endothelial,Pan_D.Endothelial,Pse_S.Endothelial,Cpl_S.Endothelial,Pbu_S.Endothelial,Asp_S.Endothelial,Hsa_S.Lymphatic_endothelia,Hsa_S.Vascular_endothelia,id,counts
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,PTPRB,21 (Endothelial)
1,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,MYCT1,20 (Endothelial)
2,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,1,1,1,1,TIE1,20 (Endothelial)
3,1,1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,KDR,20 (Endothelial)
4,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,SHE,20 (Endothelial)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,1,0,0,0,1,0,0,1,1,0,...,0,0,1,0,0,0,0,0,RGS5,6 (Endothelial)
431,0,1,0,0,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,LAMA4,6 (Endothelial)
432,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,KLF4,6 (Endothelial)
433,1,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,1,0,0,ICAM2,6 (Endothelial)


In [6]:
#Endocrine
Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[15,2]) #,13,20,16,8,17
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[12,20])#,2,9,14,10,18,7,27,28,22
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[9])#11,19,8
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[14])#,5,2,1,21,
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[9])#2,6,4,3,10,25,18,22,19
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[12])#,15,17,10
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[6,16])#,2,3,18,14,12
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[8,15,16])#,1,21,22
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[8])#6,16,17,12,25,18,27,32
Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[36])#,18,23,19,34

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[7,8,12])#,24,6,27,29
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[16,28])#
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[6])#,18,23,15,12,7,13,25,21
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[15])#,12,23,16,14
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[4])#4,9,11,10,17,21,19,23,8,5,22,15,24
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[9])#1,13,18,6,19
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[9,25])#,16,2,20,
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[6])#2,8,10,14,5
Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[20])#,4,27,9,19,13,3,21

Endocrine = {
    'Ome_I.Enteroendocrine':Ome_I,
    'Mat_I.Enteroendocrine':Mat_I,
    'Psp_I.Enteroendocrine':Psa_I,
    'Pan_I.Enteroendocrine':Pan_I,
    'Pse_I.Enteroendocrine':Pse_I,
    'Cpl_I.Enteroendocrine':Cpl_I,
    'Lre_G.Enteroendocrine':Lja_G,
    'Pbu_I.Enteroendocrine':Pbu_I,
    'Asp_I.Enteroendocrine':Asp_I,
    'Hsa_I.Enteroendocrine':Hsp_I,
    
    'Ome_D.Enteroendocrine':Ome_S,
    'Mat_S.Enteroendocrine':Mat_S,
    'Psp_S.Enteroendocrine':Psa_S,
    'Pan_D.Enteroendocrine':Pan_S,
    'Pse_S.Enteroendocrine':Pse_S,
    'Cpl_S.Enteroendocrine':Cpl_S,
    'Pbu_S.Enteroendocrine':Pbu_S,
    'Asp_S.Enteroendocrine':Asp_S, 
    'Hsa_S.Enteroendocrine':Hsp_S,
}

In [7]:
Endocrine_ai = ancestral_input_data(Endocrine,'Endocrine')
Endocrine_ai

,Ome_I.Enteroendocrine,Mat_I.Enteroendocrine,Psp_I.Enteroendocrine,Pan_I.Enteroendocrine,Pse_I.Enteroendocrine,Cpl_I.Enteroendocrine,Lre_G.Enteroendocrine,Pbu_I.Enteroendocrine,Asp_I.Enteroendocrine,Hsa_I.Enteroendocrine,...,Mat_S.Enteroendocrine,Psp_S.Enteroendocrine,Pan_D.Enteroendocrine,Pse_S.Enteroendocrine,Cpl_S.Enteroendocrine,Pbu_S.Enteroendocrine,Asp_S.Enteroendocrine,Hsa_S.Enteroendocrine,id,counts
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,PCSK1,19 (Endocrine)
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,PAX6,19 (Endocrine)
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,SCGN,19 (Endocrine)
3,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,SCG3,18 (Endocrine)
4,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,FEV,18 (Endocrine)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,1,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,CARD14,5 (Endocrine)
306,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,1,1,ARHGEF38,5 (Endocrine)
307,0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,1,0,RFX3,5 (Endocrine)
308,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,1,0,CBFA2T2,5 (Endocrine)


In [8]:
#Myeloid
#新的module
Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[9,14])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[15,6,8])
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[5])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[16,20])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[5,8])
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[5,7])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[5,9])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[2,7])
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[2,10])
Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[9])

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[4,11,13])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[3,5])
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[3])
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[6])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[12,14])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[10])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[18,4])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[7,4,9])
Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[8,23])	

Myeloid = {
    'Ome_I.Myeloid':Ome_I,
    'Mat_I.Myeloid':Mat_I,
    'Psp_I.Myeloid':Psa_I,
    'Pan_I.Myeloid':Pan_I,
    'Pse_I.Myeloid':Pse_I,
    'Cpl_I.Myeloid':Cpl_I,
    'Lre_G.Myeloid':Lja_G,
    'Pbu_I.Myeloid':Pbu_I,
    'Asp_I.Myeloid':Asp_I,
    'Hsa_I.Myeloid':Hsp_I,
    
    'Ome_D.Myeloid':Ome_S,
    'Mat_S.Myeloid':Mat_S,
    'Psp_S.Myeloid':Psa_S,
    'Pan_D.Myeloid':Pan_S,
    'Pse_S.Myeloid':Pse_S,
    'Cpl_S.Myeloid':Cpl_S,
    'Pbu_S.Myeloid':Pbu_S,
    'Asp_S.Myeloid':Asp_S, 
    'Hsa_S.Myeloid':Hsp_S,
}

In [9]:
Myeloid_ai = ancestral_input_data(Myeloid,'Myeloid')
Myeloid_ai

,Ome_I.Myeloid,Mat_I.Myeloid,Psp_I.Myeloid,Pan_I.Myeloid,Pse_I.Myeloid,Cpl_I.Myeloid,Lre_G.Myeloid,Pbu_I.Myeloid,Asp_I.Myeloid,Hsa_I.Myeloid,...,Mat_S.Myeloid,Psp_S.Myeloid,Pan_D.Myeloid,Pse_S.Myeloid,Cpl_S.Myeloid,Pbu_S.Myeloid,Asp_S.Myeloid,Hsa_S.Myeloid,id,counts
0,1,1,1,1,1,1,0,0,1,1,...,1,1,1,1,1,1,1,1,CTSC,17 (Myeloid)
1,1,0,0,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,TPP1,16 (Myeloid)
2,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,0,1,0,RNASET2,16 (Myeloid)
3,1,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,0,1,ASAH1,16 (Myeloid)
4,0,1,1,0,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,CPVL,16 (Myeloid)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,0,1,0,0,1,0,1,0,0,1,...,0,0,1,0,0,0,0,0,GLUL,5 (Myeloid)
430,0,0,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,0,1,MAP3K8,5 (Myeloid)
431,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,1,0,0,HRH2,5 (Myeloid)
432,0,0,0,0,1,1,0,0,0,0,...,1,0,0,1,0,1,0,0,CTSV,5 (Myeloid)


In [10]:
#祖先溯源版
Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[3])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[5])
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[15])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[10])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[11])
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[8])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[20,13,8])#1,20,11,8
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[9])
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[9,11])
Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[6,15])

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[19])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[14])#1,12,14
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[11])
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[8])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[6])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[8])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[8])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[15,11])
Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[7])

T_cell = {
    'Ome_I.T_cells':Ome_I,
    'Mat_I.T_cells':Mat_I,
    'Psp_I.T_cells':Psa_I,
    'Pan_I.T_cells':Pan_I,
    'Pse_I.T_cells':Pse_I,
    'Cpl_I.T_cells':Cpl_I,
    'Lre_G.T_like':Lja_G,
    'Pbu_I.T_cells':Pbu_I,
    'Asp_I.T_cells':Asp_I,
    'Hsa_I.T_NK':Hsp_I,
    
    'Ome_D.T_cells':Ome_S,
    'Mat_S.T_cells':Mat_S,
    'Psp_S.T_cells':Psa_S,
    'Pan_D.T_cells':Pan_S,
    'Pse_S.T_cells':Pse_S,
    'Cpl_S.T_cells':Cpl_S,
    'Pbu_S.T_cells':Pbu_S,
    'Asp_S.T_cells':Asp_S, 
    'Hsa_S.T_NK':Hsp_S,
}

In [11]:
T_cell_ai = ancestral_input_data(T_cell,'T_cell')
T_cell_ai

,Ome_I.T_cells,Mat_I.T_cells,Psp_I.T_cells,Pan_I.T_cells,Pse_I.T_cells,Cpl_I.T_cells,Lre_G.T_like,Pbu_I.T_cells,Asp_I.T_cells,Hsa_I.T_NK,...,Mat_S.T_cells,Psp_S.T_cells,Pan_D.T_cells,Pse_S.T_cells,Cpl_S.T_cells,Pbu_S.T_cells,Asp_S.T_cells,Hsa_S.T_NK,id,counts
0,1,1,1,1,1,1,0,1,1,1,...,0,1,1,1,1,1,1,1,FGD3,17 (T_cell)
1,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,0,1,1,1,BCL11B,17 (T_cell)
2,1,1,1,1,1,1,0,1,1,1,...,1,0,1,1,1,1,1,1,CD247,16 (T_cell)
3,1,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,1,1,LCK,16 (T_cell)
4,1,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,1,1,ZAP70,16 (T_cell)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0,1,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,1,0,RORA,5 (T_cell)
230,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,1,RHOH,5 (T_cell)
231,1,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,PTPN7,5 (T_cell)
232,0,0,0,0,0,1,0,1,0,1,...,0,0,0,1,1,0,0,0,LNPEP,5 (T_cell)


In [12]:
#Mesenchymal
Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[6,7,12,19])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[4])
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[10,13])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[6,11])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[4,3,14])
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[2])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[22,1,4,12])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[5])
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[19,3])

Hsp_I_SMC = find_specfic_cluster(csv_data_dic['Hsp.I'],[14,34,30,16,31])
Hsp_I_L = find_specfic_cluster(csv_data_dic['Hsp.I'],[29,35,26])
Hsp_I_P = find_specfic_cluster(csv_data_dic['Hsp.I'],[10,26])
Hsp_I_F = find_specfic_cluster(csv_data_dic['Hsp.I'],[2])
Hsp_I_MF = find_specfic_cluster(csv_data_dic['Hsp.I'],[26])

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[9,17])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[13,15])
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[8])
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[2])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[3])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[3])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[7,11])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[3,1])

Hsp_S_SMC = find_specfic_cluster(csv_data_dic['Hsp.S'],[28]) #?
Hsp_S_L = find_specfic_cluster(csv_data_dic['Hsp.S'],[27,15])
Hsp_S_P = find_specfic_cluster(csv_data_dic['Hsp.S'],[17])
Hsp_S_F = find_specfic_cluster(csv_data_dic['Hsp.S'],[4,28])
Hsp_S_MF = find_specfic_cluster(csv_data_dic['Hsp.S'],[4,28])

Mesenchymal = {
    'Ome_I.Mesenchymal':Ome_I,
    'Mat_I.Mesenchymal':Mat_I,
    'Psp_I.Mesenchymal':Psa_I,
    'Pan_I.Mesenchymal':Pan_I,
    'Pse_I.Mesenchymal':Pse_I,
    'Cpl_I.Mesenchymal':Cpl_I,
    'Lre_G.Mesenchymal':Lja_G,
    'Pbu_I.Mesenchymal':Pbu_I,
    'Asp_I.Mesenchymal':Asp_I,

    'Hsa_I.Smooth_muscle':Hsp_I_SMC,
    'Hsa_I.Lymphoid_stromal':Hsp_I_L,
    'Hsa_I.Fibroblast':Hsp_I_F,
    'Hsa_I.Pericyte':Hsp_I_P,
    'Hsa_I.Myofibroblast':Hsp_I_MF,
    
    'Ome_D.Mesenchymal':Ome_S,
    'Mat_S.Mesenchymal':Mat_S,
    'Psp_S.Mesenchymal':Psa_S,
    'Pan_D.Mesenchymal':Pan_S,
    'Pse_S.Mesenchymal':Pse_S,
    'Cpl_S.Mesenchymal':Cpl_S,
    'Pbu_S.Mesenchymal':Pbu_S,
    'Asp_S.Mesenchymal':Asp_S,

    'Hsa_S.Smooth_muscle':Hsp_S_SMC,
    'Hsa_S.Lymphoid_stromal':Hsp_S_L,
    'Hsa_S.Fibroblast':Hsp_S_F,
    'Hsa_S.Pericyte':Hsp_S_P,
    'Hsa_S.Myofibroblast':Hsp_S_MF,
}

In [13]:
Mesenchymal_ai = ancestral_input_data(Mesenchymal,'Mesenchymal')
Mesenchymal_ai

,Ome_I.Mesenchymal,Mat_I.Mesenchymal,Psp_I.Mesenchymal,Pan_I.Mesenchymal,Pse_I.Mesenchymal,Cpl_I.Mesenchymal,Lre_G.Mesenchymal,Pbu_I.Mesenchymal,Asp_I.Mesenchymal,Hsa_I.Smooth_muscle,...,Cpl_S.Mesenchymal,Pbu_S.Mesenchymal,Asp_S.Mesenchymal,Hsa_S.Smooth_muscle,Hsa_S.Lymphoid_stromal,Hsa_S.Fibroblast,Hsa_S.Pericyte,Hsa_S.Myofibroblast,id,counts
0,1,1,1,1,1,1,1,1,1,0,...,1,1,1,0,0,1,0,1,COL6A1,20 (Mesenchymal)
1,1,1,1,1,1,1,1,1,1,0,...,1,1,1,0,0,1,0,1,COL6A2,20 (Mesenchymal)
2,1,1,1,1,1,1,0,1,1,0,...,1,1,1,0,0,1,0,1,COL1A1,19 (Mesenchymal)
3,0,1,1,1,1,1,1,1,1,0,...,1,1,1,0,0,1,0,1,FBN1,18 (Mesenchymal)
4,1,1,1,1,1,1,0,0,1,0,...,1,1,1,0,0,1,0,1,COL5A2,18 (Mesenchymal)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0,0,1,0,1,1,0,0,0,1,...,0,1,0,0,0,1,0,1,TPM2,7 (Mesenchymal)
345,0,0,0,1,0,0,1,0,0,1,...,1,0,0,0,0,0,1,0,S1PR3,7 (Mesenchymal)
346,1,1,0,0,1,0,0,1,1,0,...,0,1,1,0,0,0,0,0,TNFSF10,7 (Mesenchymal)
347,1,0,0,0,1,1,0,0,0,0,...,1,0,0,0,0,1,0,1,PDLIM4,7 (Mesenchymal)


In [14]:
#Enterocyte

Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[4])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[1,7])
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[3])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[1,21])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[6])
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[4])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[17,12])
Lja_G_like = find_specfic_cluster(csv_data_dic['Lja.G'],[3])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[6,18])
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[1,4,15])
Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[12,11,21])
Hsp_I_M = find_specfic_cluster(csv_data_dic['Hsp.I'],[5,4,26])

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[3,14])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[1,19])
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[20,21])
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[14,17])
# Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[])
# Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[1,2])
# Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[])
Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[6,12])

Enterocyte = {
    'Ome_I.Enterocytes':Ome_I,
    'Mat_I.Enterocytes':Mat_I,
    'Psp_I.Enterocytes':Psa_I,
    'Pan_I.Enterocytes':Pan_I,
    'Pse_I.Enterocytes':Pse_I,
    'Cpl_I.Enterocytes':Cpl_I,
    'Lre_G.Enterocytes':Lja_G,
    'Lre_G.Enterocytes_like':Lja_G_like,
    'Pbu_I.Enterocytes':Pbu_I,
    'Asp_I.Enterocytes':Asp_I,
    'Hsa_I.Enterocytes':Hsp_I,
    'Hsa_I.Microfold':Hsp_I_M,
    
    'Ome_D.Enterocytes':Ome_S,
    'Mat_S.Enterocytes':Mat_S,
    'Psp_S.Enterocytes':Psa_S,
    'Pan_D.Enterocytes':Pan_S,
    # 'Pse_S.':Pse_S,
    # 'Cpl_S.':Cpl_S,
    'Pbu_S.Enterocytes':Pbu_S,
    # 'Asp_S.':Asp_S, 
    'Hsa_S.Enterocytes':Hsp_S,
}

In [15]:
Enterocyte_ai = ancestral_input_data(Enterocyte,'Enterocyte')
Enterocyte_ai

,Ome_I.Enterocytes,Mat_I.Enterocytes,Psp_I.Enterocytes,Pan_I.Enterocytes,Pse_I.Enterocytes,Cpl_I.Enterocytes,Lre_G.Enterocytes,Lre_G.Enterocytes_like,Pbu_I.Enterocytes,Asp_I.Enterocytes,Hsa_I.Enterocytes,Hsa_I.Microfold,Ome_D.Enterocytes,Mat_S.Enterocytes,Psp_S.Enterocytes,Pan_D.Enterocytes,Pbu_S.Enterocytes,Hsa_S.Enterocytes,id,counts
0,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,ACE2,16 (Enterocyte)
1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,SLC15A1,15 (Enterocyte)
2,1,1,1,1,1,1,0,0,1,1,0,1,1,1,1,1,1,1,FABP1,15 (Enterocyte)
3,1,1,1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,1,ASAH2,15 (Enterocyte)
4,1,1,1,1,1,1,0,1,1,1,0,1,1,0,1,1,1,1,MGAM,15 (Enterocyte)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,1,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,SLC9A3R1,5 (Enterocyte)
335,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,ACO1,5 (Enterocyte)
336,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,IDH1,5 (Enterocyte)
337,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,IYD,5 (Enterocyte)


In [16]:
#B_cell
#New module
# # Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[26,1])#?
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[1,12])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[8,7,19,12])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[16])
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[1,25])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[1])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[21])#?
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[25,29])
Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[8,17])	

# # Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[1])#?
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[15])
# # Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[1,16,8,19])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[2])#?
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[17])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[17])
Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[26])

B_cell = {
    # 'Ome_I.':Ome_I,
    'Mat_I.B_cells':Mat_I,
    'Psp_I.B_cells':Psa_I,
    'Pan_I.B_cells':Pan_I,
    'Pse_I.B_cells':Pse_I,
    'Cpl_I.B_cells':Cpl_I,
    'Lre_G.B_like':Lja_G,
    'Pbu_I.B_cells':Pbu_I,
    'Asp_I.B_cells':Asp_I,
    'Hsa_I.B_plasma':Hsp_I,
    
    # 'Ome_D.':Ome_S,
    'Mat_S.B_cells':Mat_S,
    'Psp_S.B_cells':Psa_S,
    # 'Pan_D.':Pan_S,
    'Pse_S.B_cells':Pse_S,
    'Cpl_S.B_cells':Cpl_S,
    'Pbu_S.B_cells':Pbu_S,
    'Asp_S.B_cells':Asp_S, 
    'Hsa_S.B_plasma':Hsp_S,
}

In [17]:
B_cell_ai = ancestral_input_data(B_cell,'B_cell')
B_cell_ai

,Mat_I.B_cells,Psp_I.B_cells,Pan_I.B_cells,Pse_I.B_cells,Cpl_I.B_cells,Lre_G.B_like,Pbu_I.B_cells,Asp_I.B_cells,Hsa_I.B_plasma,Mat_S.B_cells,Psp_S.B_cells,Pse_S.B_cells,Cpl_S.B_cells,Pbu_S.B_cells,Asp_S.B_cells,Hsa_S.B_plasma,id,counts
0,0,0,1,1,0,0,0,1,1,0,1,1,0,1,1,1,TNFRSF17,9 (B_cell)
1,0,1,0,1,1,0,0,1,1,0,1,1,0,1,0,1,TNFRSF13B,9 (B_cell)
2,0,0,1,0,0,0,0,1,1,1,1,0,0,1,1,1,PRDX4,8 (B_cell)
3,0,1,0,1,1,0,0,1,1,0,1,1,0,1,0,0,CD79B,8 (B_cell)
4,0,1,1,1,0,0,0,1,1,0,1,1,0,1,0,0,PAX5,8 (B_cell)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,NPM1,4 (B_cell)
194,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,BANK1,4 (B_cell)
195,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,SSR4,4 (B_cell)
196,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,YBX3,4 (B_cell)


In [18]:
#Mucus_Goblet
Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[17])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[2])
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[7,8,21])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[5,13])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[12,19])
Cpl_I_Glike = find_specfic_cluster(csv_data_dic['Cpl.I'],[22,17,6,13,21])
Cpl_I_G = find_specfic_cluster(csv_data_dic['Cpl.I'],[10])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[18,14])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[11])
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[12])
Hsp_I_G = find_specfic_cluster(csv_data_dic['Hsp.I'],[23])
Hsp_I_P = find_specfic_cluster(csv_data_dic['Hsp.I'],[34])#?
Hsp_I_F = find_specfic_cluster(csv_data_dic['Hsp.I'],[12,27])#?			

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[29])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[2,7,9,12,17,31])
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[7])
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[7,9])
Pse_S_F = find_specfic_cluster(csv_data_dic['Pse.S'],[22]) #23 Feveolar
Pse_S_M = find_specfic_cluster(csv_data_dic['Pse.S'],[22])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[23,21,6,15,11])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[15,22])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[8])
Hsp_S_G = find_specfic_cluster(csv_data_dic['Hsp.S'],[10,21])#11,12,5,18
Hsp_S_F = find_specfic_cluster(csv_data_dic['Hsp.S'],[10,13])#11,12,5,18
# Hsp_S_M = find_specfic_cluster(csv_data_dic['Hsp.S'],[21,10,1])#11,12,5,18

Mucus_Goblet = {
    'Ome_I.Goblet':Ome_I,
    'Mat_I.Goblet':Mat_I,
    'Psp_I.Goblet':Psa_I,
    'Pan_I.Goblet':Pan_I,
    'Pse_I.Goblet':Pse_I,
    'Cpl_I.Goblet_like':Cpl_I_Glike,
    'Cpl_I.Goblet':Cpl_I_G,
    'Lre_G.Goblet':Lja_G,
    'Pbu_I.Goblet':Pbu_I,
    'Asp_I.Goblet':Asp_I,
    'Hsa_I.Goblet':Hsp_I_G,
    'Hsa_I.Paneth':Hsp_I_P,
    'Hsa_I.Foveolar':Hsp_I_F,
    
    'Ome_D.Mucus':Ome_S,
    'Mat_S.Mucus':Mat_S,
    'Psp_S.Mucus':Psa_S,
    'Pan_D.Mucus':Pan_S,

    'Pse_S.Foveolar':Pse_S_F,
    'Pse_S.Mucus':Pse_S_M,

    'Cpl_S.Mucus':Cpl_S,
    'Pbu_S.Mucus':Pbu_S,
    'Asp_S.Mucus':Asp_S, 
    'Hsa_S.Foveolar':Hsp_S_F,
    'Hsa_S.Goblet':Hsp_S_G,
    # 'Hsa_S.Paneth':Hsp_S_M,
}

In [19]:
Mucus_Goblet_ai = ancestral_input_data(Mucus_Goblet,'Mucus_Goblet')
Mucus_Goblet_ai

,Ome_I.Goblet,Mat_I.Goblet,Psp_I.Goblet,Pan_I.Goblet,Pse_I.Goblet,Cpl_I.Goblet_like,Cpl_I.Goblet,Lre_G.Goblet,Pbu_I.Goblet,Asp_I.Goblet,...,Pan_D.Mucus,Pse_S.Foveolar,Pse_S.Mucus,Cpl_S.Mucus,Pbu_S.Mucus,Asp_S.Mucus,Hsa_S.Foveolar,Hsa_S.Goblet,id,counts
0,1,1,1,1,0,0,1,0,1,1,...,1,1,1,1,0,0,1,1,STARD10,16 (Mucus_Goblet)
1,1,1,1,1,1,0,1,0,1,1,...,1,1,1,1,1,0,0,1,SPDEF,16 (Mucus_Goblet)
2,1,1,1,0,0,0,1,0,1,1,...,0,1,1,0,1,1,1,1,FER1L6,15 (Mucus_Goblet)
3,0,1,1,1,0,0,1,0,1,1,...,0,1,1,1,0,1,1,1,BCAS1,15 (Mucus_Goblet)
4,1,1,0,1,0,0,1,0,0,0,...,1,1,1,1,0,1,1,1,GALNT3,14 (Mucus_Goblet)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,1,0,0,1,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,CASC4,6 (Mucus_Goblet)
142,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,1,DDR1,6 (Mucus_Goblet)
143,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,1,MLPH,6 (Mucus_Goblet)
144,0,0,0,0,0,0,0,0,1,0,...,0,1,1,0,0,0,1,1,TMEM38A,6 (Mucus_Goblet)


In [20]:
#Ciliated
# Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[])
# Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[])
Psa_I = find_specfic_cluster(csv_data_dic['Psa.I'],[2])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[2])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[2])
# Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[])
Lja_G = find_specfic_cluster(csv_data_dic['Lja.G'],[2])
# Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[])
# Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[])
# Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[])

# Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[4])
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[1])
Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[1,4])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[5])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[1])
# Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[2,6])
# Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[])

Ciliated = {
    # 'Ome_I.':Ome_I,
    # 'Mat_I.':Mat_I,
    'Psp_I.Ciliated':Psa_I,
    'Pan_I.Ciliated':Pan_I,
    'Pse_I.Ciliated':Pse_I,
    # 'Cpl_I.':Cpl_I,
    'Lre_G.Ciliated':Lja_G,
    # 'Pbu_I.':Pbu_I,
    # 'Asp_I.':Asp_I,
    # 'Hsa_I.':Hsp_I,
    
    # 'Ome_D.':Ome_S,
    'Mat_S.Ciliated':Mat_S,
    'Psp_S.Ciliated':Psa_S,
    'Pan_D.Ciliated':Pan_S,
    'Pse_S.Ciliated':Pse_S,
    'Cpl_S.Ciliated':Cpl_S,
    # 'Pbu_S.':Pbu_S,
    'Asp_S.Ciliated':Asp_S, 
    # 'Hsa_S.':Hsp_S,
}

In [21]:
Ciliated_ai = ancestral_input_data(Ciliated,'Ciliated')
Ciliated_ai

,Psp_I.Ciliated,Pan_I.Ciliated,Pse_I.Ciliated,Lre_G.Ciliated,Mat_S.Ciliated,Psp_S.Ciliated,Pan_D.Ciliated,Pse_S.Ciliated,Cpl_S.Ciliated,Asp_S.Ciliated,id,counts
0,1,1,1,1,1,1,1,1,1,1,CCDC173,10 (Ciliated)
1,1,1,1,1,1,1,1,1,1,1,MOK,10 (Ciliated)
2,1,1,1,1,1,1,1,1,1,1,MEIG1,10 (Ciliated)
3,1,1,1,1,1,1,1,1,1,1,SPATA18,10 (Ciliated)
4,1,1,1,1,1,1,1,1,1,1,TRAF3IP1,10 (Ciliated)
...,...,...,...,...,...,...,...,...,...,...,...,...
513,0,0,0,1,0,0,0,0,1,1,TXNDC9,3 (Ciliated)
514,1,0,0,0,0,1,0,0,1,0,DYNLL1,3 (Ciliated)
515,0,1,0,0,0,0,1,0,0,1,EFCAB10,3 (Ciliated)
516,1,0,0,0,0,1,0,0,0,1,FOXP2,3 (Ciliated)


In [22]:
#Tuft

Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[10])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[22])
Psp_I = find_specfic_cluster(csv_data_dic['Psa.I'],[22])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[5])#？
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[23])
Cpl_I = find_specfic_cluster(csv_data_dic['Cpl.I'],[11,14])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[22]) #？
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[6])
Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[27,33])#？

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[21])
Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[24,32])
Psp_S = find_specfic_cluster(csv_data_dic['Psa.S'],[12])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[11])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[29])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[12,18,10])

Tuft = {
    'Ome_I.Tuft':Ome_I,
    'Mat_I.Tuft':Mat_I,
    'Psp_I.Tuft':Psp_I,
    'Pan_I.Tuft':Pan_I,
    'Pse_I.Tuft':Pse_I,
    'Cpl_I.Tuft':Cpl_I,
    'Pbu_I.Tuft':Pbu_I,
    'Asp_I.Tuft':Asp_I,
    'Hsa_I.Tuft':Hsp_I,
    
    'Ome_D.Tuft':Ome_S,
    'Mat_S.Tuft':Mat_S,
    'Psp_S.Tuft':Psp_S,
    'Pse_S.Tuft':Pse_S,
    'Pbu_S.Tuft':Pbu_S,
    'Asp_S.Tuft':Asp_S, 
}

In [23]:
Tuft_ai = ancestral_input_data(Tuft,'Tuft')
Tuft_ai

,Ome_I.Tuft,Mat_I.Tuft,Psp_I.Tuft,Pan_I.Tuft,Pse_I.Tuft,Cpl_I.Tuft,Pbu_I.Tuft,Asp_I.Tuft,Hsa_I.Tuft,Ome_D.Tuft,Mat_S.Tuft,Psp_S.Tuft,Pse_S.Tuft,Pbu_S.Tuft,Asp_S.Tuft,id,counts
0,1,1,0,0,1,1,0,1,0,1,1,0,1,1,1,LTC4S,10 (Tuft)
1,1,1,0,0,1,1,0,1,0,1,1,1,1,0,1,POU2F3,10 (Tuft)
2,1,1,0,0,1,0,0,0,0,1,1,1,1,1,1,BANK1,9 (Tuft)
3,1,1,0,0,0,0,0,1,0,1,1,1,0,1,1,CISD2,8 (Tuft)
4,0,1,0,0,1,1,0,1,0,0,1,0,1,1,1,C11ORF53,8 (Tuft)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,STAM2,4 (Tuft)
75,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,PTPRH,4 (Tuft)
76,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,TNFRSF11B,4 (Tuft)
77,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,RAB27B,4 (Tuft)


In [24]:
#BEST4
Ome_I = find_specfic_cluster(csv_data_dic['Ome.I'],[13])
Mat_I = find_specfic_cluster(csv_data_dic['Mat.I'],[9])
Pan_I = find_specfic_cluster(csv_data_dic['Pan.I'],[13,15])
Pse_I = find_specfic_cluster(csv_data_dic['Pse.I'],[19])
Pbu_I = find_specfic_cluster(csv_data_dic['Pbu.I'],[18,10])
Asp_I = find_specfic_cluster(csv_data_dic['Asp.I'],[17])
Hsp_I = find_specfic_cluster(csv_data_dic['Hsp.I'],[19])

Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[35])
Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[10])#？

BEST4 = {
    'Ome_I.BEST4':Ome_I,
    'Mat_I.BEST4_like':Mat_I,
    'Pan_I.BEST4':Pan_I,
    'Pse_I.BEST4':Pse_I,
    'Pbu_I.BEST4':Pbu_I,
    'Asp_I.BEST4':Asp_I,
    'Hsa_I.BEST4':Hsp_I,
    
    'Ome_D.BEST4':Ome_S,
    'Hsa_S.BEST4':Hsp_S,
}

In [25]:
BEST4_ai = ancestral_input_data(BEST4,'BEST4')
BEST4_ai

,Ome_I.BEST4,Mat_I.BEST4_like,Pan_I.BEST4,Pse_I.BEST4,Pbu_I.BEST4,Asp_I.BEST4,Hsa_I.BEST4,Ome_D.BEST4,Hsa_S.BEST4,id,counts
0,0,0,0,1,1,1,1,0,1,DHCR24,5 (BEST4)
1,0,0,0,0,1,0,1,1,1,GALNT7,4 (BEST4)
2,1,0,0,1,0,1,1,0,0,PDE3A,4 (BEST4)
3,0,0,0,0,1,1,1,0,1,CYP51A1,4 (BEST4)
4,1,1,0,0,0,1,1,0,0,PRLR,4 (BEST4)
5,1,1,0,0,0,0,1,0,1,DSP,4 (BEST4)
6,1,1,0,0,0,0,1,0,1,OCLN,4 (BEST4)
7,1,1,0,0,0,1,0,0,1,SLC37A1,4 (BEST4)
8,1,1,0,0,0,0,1,0,1,MARVELD2,4 (BEST4)
9,1,0,0,1,0,1,0,1,0,PBX3,4 (BEST4)


In [26]:
#Oxynticopeptic
# # Ome_S = find_specfic_cluster(csv_data_dic['Ome.S'],[])
# Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[13,1,12,19])
# Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[14,17])
# # Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[])
# Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[13,26,20])
# Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[5])
# Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[28])
# Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[8,1,16])
# Hsp_S = find_specfic_cluster(csv_data_dic['Hsp.S'],[24,3,21,10])

Mat_S = find_specfic_cluster(csv_data_dic['Mat.S'],[11,30])
Psa_S = find_specfic_cluster(csv_data_dic['Psa.S'],[14,17])
# Pan_S = find_specfic_cluster(csv_data_dic['Pan.S'],[])
Pse_S = find_specfic_cluster(csv_data_dic['Pse.S'],[13])
Pse_S_like = find_specfic_cluster(csv_data_dic['Pse.S'],[13])
Cpl_S = find_specfic_cluster(csv_data_dic['Cpl.S'],[5])
Pbu_S = find_specfic_cluster(csv_data_dic['Pbu.S'],[28])
Asp_S = find_specfic_cluster(csv_data_dic['Asp.S'],[1,16])
Hsp_S_C = find_specfic_cluster(csv_data_dic['Hsp.S'],[21])
Hsp_S_P = find_specfic_cluster(csv_data_dic['Hsp.S'],[24])
Hsp_S_MN = find_specfic_cluster(csv_data_dic['Hsp.S'],[21,10])#？
Hsp_I_MN = find_specfic_cluster(csv_data_dic['Hsp.S'],[34,33])#？

Oxynticopeptic = {   
    # 'Ome_S':Ome_S,
    'Mat_S.Oxynticopeptic':Mat_S,
    'Psp_S.Oxynticopeptic':Psa_S,
    # 'Pan_S':Pan_S,
    'Pse_S.Oxynticopeptic':Pse_S,
    'Pse_S.Oxynticopeptic_like':Pse_S_like,
    'Cpl_S.Oxynticopeptic':Cpl_S,
    'Pbu_S.Oxynticopeptic':Pbu_S,
    'Asp_S.Oxynticopeptic':Asp_S, 
    'Hsa_S.Chief':Hsp_S_C,
    'Hsa_S.Parietal':Hsp_S_P,
    'Hsa_S.Mucous':Hsp_S_MN,
    'Hsa_I.Mucous':Hsp_I_MN,
}

In [27]:
Oxynticopeptic_ai = ancestral_input_data(Oxynticopeptic,'Oxynticopeptic')
Oxynticopeptic_ai

,Mat_S.Oxynticopeptic,Psp_S.Oxynticopeptic,Pse_S.Oxynticopeptic,Pse_S.Oxynticopeptic_like,Cpl_S.Oxynticopeptic,Pbu_S.Oxynticopeptic,Asp_S.Oxynticopeptic,Hsa_S.Chief,Hsa_S.Parietal,Hsa_S.Mucous,Hsa_I.Mucous,id,counts
0,1,1,1,1,1,0,1,1,0,1,0,ALDH6A1,8 (Oxynticopeptic)
1,1,1,1,1,1,1,1,0,1,0,0,IDH2,8 (Oxynticopeptic)
2,1,1,1,1,1,1,1,0,1,0,0,ESRRG,8 (Oxynticopeptic)
3,0,1,1,1,1,1,1,1,0,1,0,PGC,8 (Oxynticopeptic)
4,0,1,1,1,1,1,1,0,1,0,0,ATP4B,7 (Oxynticopeptic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,0,0,0,0,1,0,0,1,0,1,0,SH3YL1,3 (Oxynticopeptic)
235,1,0,0,0,1,0,1,0,0,0,0,CYC1,3 (Oxynticopeptic)
236,0,0,0,0,0,1,0,1,0,1,0,RGMB,3 (Oxynticopeptic)
237,0,1,0,0,0,1,0,0,1,0,0,KCNQ1,3 (Oxynticopeptic)


In [28]:
import pandas as pd
import numpy as np

def replace_nan_with_int_zero(df):

    numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
    
    # 将数值列中的 NaN 替换为 0
    df[numeric_cols] = df[numeric_cols].fillna(0)
    
    # 将所有数值列转换为整数类型
    for col in numeric_cols:
        # 检查列是否包含浮点数（可能因NaN转换产生）
        if pd.api.types.is_float_dtype(df[col]):
            # 确保所有值都是整数（或可以安全转换为整数）
            if (df[col] % 1 == 0).all():
                df[col] = df[col].astype(int)
    
    return df

In [29]:
df_list = [Endocrine_ai,Enterocyte_ai,Endothelial_ai,Mucus_Goblet_ai,Myeloid_ai,Mesenchymal_ai,T_cell_ai,B_cell_ai,Oxynticopeptic_ai,Tuft_ai,BEST4_ai,Ciliated_ai]

merged_df = pd.concat(df_list, ignore_index=True)
df = replace_nan_with_int_zero(merged_df)

In [30]:
df

,Ome_I.Enteroendocrine,Mat_I.Enteroendocrine,Psp_I.Enteroendocrine,Pan_I.Enteroendocrine,Pse_I.Enteroendocrine,Cpl_I.Enteroendocrine,Lre_G.Enteroendocrine,Pbu_I.Enteroendocrine,Asp_I.Enteroendocrine,Hsa_I.Enteroendocrine,...,Psp_I.Ciliated,Pan_I.Ciliated,Pse_I.Ciliated,Lre_G.Ciliated,Mat_S.Ciliated,Psp_S.Ciliated,Pan_D.Ciliated,Pse_S.Ciliated,Cpl_S.Ciliated,Asp_S.Ciliated
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3331,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
3332,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
3333,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
3334,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1


In [31]:
df1 = df.groupby('id').sum().reset_index()

In [32]:
df1

,id,Ome_I.Enteroendocrine,Mat_I.Enteroendocrine,Psp_I.Enteroendocrine,Pan_I.Enteroendocrine,Pse_I.Enteroendocrine,Cpl_I.Enteroendocrine,Lre_G.Enteroendocrine,Pbu_I.Enteroendocrine,Asp_I.Enteroendocrine,...,Psp_I.Ciliated,Pan_I.Ciliated,Pse_I.Ciliated,Lre_G.Ciliated,Mat_S.Ciliated,Psp_S.Ciliated,Pan_D.Ciliated,Pse_S.Ciliated,Cpl_S.Ciliated,Asp_S.Ciliated
0,A1CF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AADAC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ABAT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ABCA1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ABCC12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142,shtn2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3143,stk17al,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3144,tdh,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3145,zgc:195282,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# df1.to_csv('D:/project/digestion_SI/ancestral_reconstruction/Alltest_20250918/All_module_gene(20250922).v7.0.2.csv',index = False)

In [34]:
T_cell_ai

,Ome_I.T_cells,Mat_I.T_cells,Psp_I.T_cells,Pan_I.T_cells,Pse_I.T_cells,Cpl_I.T_cells,Lre_G.T_like,Pbu_I.T_cells,Asp_I.T_cells,Hsa_I.T_NK,...,Mat_S.T_cells,Psp_S.T_cells,Pan_D.T_cells,Pse_S.T_cells,Cpl_S.T_cells,Pbu_S.T_cells,Asp_S.T_cells,Hsa_S.T_NK,id,counts
0,1,1,1,1,1,1,0,1,1,1,...,0,1,1,1,1,1,1,1,FGD3,17 (T_cell)
1,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,0,1,1,1,BCL11B,17 (T_cell)
2,1,1,1,1,1,1,0,1,1,1,...,1,0,1,1,1,1,1,1,CD247,16 (T_cell)
3,1,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,1,1,LCK,16 (T_cell)
4,1,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,1,1,ZAP70,16 (T_cell)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,0,1,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,1,0,RORA,5 (T_cell)
230,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,1,RHOH,5 (T_cell)
231,1,0,0,0,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,PTPN7,5 (T_cell)
232,0,0,0,0,0,1,0,1,0,1,...,0,0,0,1,1,0,0,0,LNPEP,5 (T_cell)


In [35]:
T_cell_ai.to_csv('D:/project/digestion_SI/ancestral_reconstruction/Ttest_20250924/T_cell_gene(0924).csv',index = False)